#1 INICIALIZAR BIBLIOTECAS inicializar();

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf
import datetime
from datetime import date
from pandas.tseries.offsets import BDay
import winsound
from time import time
from dateutil.relativedelta import relativedelta
yf.pdr_override()


In [14]:
#Ler csv
def lercsv(nome):
    pd.set_option('display.max_rows', 10)

    df = pd.DataFrame();

    df = pd.read_csv(nome)

    df.columns = ['tickers']

    #df = df.melt(id_vars=['empresa'])

    #df.drop(columns=['variable','empresa'],inplace=True)

    df.dropna(inplace=True)

    df= df.sort_values('tickers')

    df.index = range(df.shape[0])

    return df

#baixar dados diarios do yahoofinance
# df,aux = baixardados(ndias)
def baixardados(df,ndias):
    aux = []
    #baixar os dados do yahoofinance ultimos 15 dias uteis

    start = date.today() - BDay(ndias)
    for i in range(df.shape[0]):
        aux.append(pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=start));

    df.insert(1, 'dadosd', aux, allow_duplicates=True);

    print('======= ACABOU ===========')
    return df,aux

#baixar dados semanais do yahoofinance
# df,auxwk = baixardadoswk(df,nwk)
def baixardadoswk(df,nwk):
    aux = []
    start = date.today() - relativedelta(weeks=nwk)
    for i in range(df.shape[0]):
        aux.append(pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=start,interval="1wk"));
        aux[i].dropna(inplace=True)
        if aux[i].index[aux[i].shape[0]-1] != (aux[i].index[aux[i].shape[0]-2] + datetime.timedelta(days=7)):
            aux[i].drop(index=aux[i].index[-1], axis=0, inplace=True)
    df.insert(2, 'dadoswe', aux, allow_duplicates=True);

    print('======= ACABOU ===========')
    return df,aux

#baixar dados mensais do yahoofinance
#df,auxmo = baixardadosmo(df,nmo)
def baixardadosmo(df,nmo):
    aux = []
    start = date.today() - relativedelta(months=nmo)
    for i in range(df.shape[0]):
        aux.append(pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=start,interval="1mo"));
        aux[i].dropna(inplace=True)
        if aux[i].index[aux[i].shape[0]-1] != (aux[i].index[aux[i].shape[0]-2] + datetime.timedelta(days=30)):
            aux[i].drop(index=aux[i].index[-1], axis=0, inplace=True)
    df.insert(3, 'dadosmo', aux, allow_duplicates=True);

    print('======= ACABOU ===========')
    return df,aux

# salvar data frame
def salvardf():
    df.to_hdf('df.h5', 'obj1');
    return

# carregar data frame
#df,aux,auxwk,auxmo = carregardf();
def carregardf():
    df = pd.DataFrame();
    aux = [];
    auxwk = [];
    auxmo = [];
    df = pd.read_hdf('df.h5', 'obj1');
    for i in range(df.shape[0]-1,-1,-1):
        if df.iloc[i,1].shape[0] <= 3:
            df = df.drop(df.index[i])
    df.index = range(df.shape[0])
    for i in range(df.shape[0]):
        aux.append(df.iloc[i,1])
        auxwk.append(df.iloc[i,2])
        auxmo.append(df.iloc[i,3])
    return(df,aux,auxwk,auxmo)

# atualização diária
# aux = atualizacaodiaria(df,aux)
def atualizacaodiaria(df,aux):

    for i in range(df.shape[0]):
        df2 = pd.DataFrame();
        if (max(aux[i].index) == (date.today() - BDay(1))):
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=date.today()-BDay(1));
            aux[i] = pd.concat([aux[i],df2]);
        else:
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=max(aux[i].index)-BDay(1));
            aux[i] = pd.concat([aux[i],df2]);
            
        aux[i] = aux[i][~aux[i].index.duplicated(keep='last')]

    return(aux)

#atualização semanal
# auxwk = atualizacaosemanal(df,auxwk)
def atualizacaosemanal(df,aux):
    
    for i in range(df.shape[0]):
        df2 = pd.DataFrame();
        if (max(aux[i].index) == (date.today() - datetime.timedelta(days=7))):
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=date.today() - datetime.timedelta(days=7),interval="1wk");
            aux[i] = pd.concat([aux[i],df2]);
        else:
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=max(aux[i].index)- datetime.timedelta(days=7),interval="1wk");
            aux[i] = pd.concat([aux[i],df2]);
            
        aux[i] = aux[i][~aux[i].index.duplicated(keep='last')]

    return(aux)

#atualização mensal
# auxmo = atualizacaomensal(df,auxmo)
def atualizacaomensal(df,aux):
    
    for i in range(df.shape[0]):
        df2 = pd.DataFrame();
        if (max(aux[i].index) == (date.today() - datetime.timedelta(days=30))):
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=date.today() - datetime.timedelta(days=30),interval="1mo");
            aux[i] = pd.concat([aux[i],df2]);
        else:
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=max(aux[i].index)- datetime.timedelta(days=30),interval="1mo");
            aux[i] = pd.concat([aux[i],df2]);
            
        aux[i] = aux[i][~aux[i].index.duplicated(keep='last')]

    return(aux)

#TOPOS E FUNDOS
# df,aux = tendencia(df,aux)

def tendencia(df,aux):
    #topos
    for i in range(df.shape[0]):
        aux[i]['TOPO'] = ''
        j=0;
        for k in aux[i].index:
            if j == aux[i].shape[0]-1:
                break
            else:
                aux[i].loc[k,'TOPO'] = np.where((aux[i].High[j-1] < aux[i].High[j] > aux[i].High[j+1]),1,0)
                j += 1;
            
        aux[i].at[aux[i].index[-1],'TOPO'] = np.array(0)
        aux[i].at[aux[i].index[0],'TOPO'] = np.array(0)
    
    #fundos    
    for i in range(df.shape[0]):
        aux[i]['FUNDO'] = ''
        j=0;
        for k in aux[i].index:
            if j == aux[i].shape[0]-1:
                break
            else:
                aux[i].loc[k,'FUNDO'] = np.where((aux[i].Low[j-1] > aux[i].Low[j] < aux[i].Low[j+1]),1,0)
                j +=1;
        aux[i].at[aux[i].index[-1],'FUNDO'] = np.array(0)
        aux[i].at[aux[i].index[0],'FUNDO'] = np.array(0)
        
    #lista de topos
    topos = []
    for i in range(df.shape[0]):
        inferno = []
        for j in aux[i].index:
            if int(aux[i].at[j,'TOPO']) == 1:
                inferno.append(aux[i].High[j]);
        topos.append(inferno)
    
    #lista de fundos
    fundos = []
    for i in range(df.shape[0]):
        inferno = []
        for j in aux[i].index:
            if int(aux[i].at[j,'FUNDO']) == 1:
                inferno.append(aux[i].Low[j]);
        fundos.append(inferno)
    #regras de tendencia    
    for i in range(0,len(topos)-1):
        if ((len(fundos[i])>2) & (len(topos[i])>2)):
            if (topos[i][-1] > topos[i][-2]) & (fundos[i][-1] > fundos[i][-2]):
                df['TENDENCIA'][i] = 'ALTA'
            if (topos[i][-1] < topos[i][-2]) & (fundos[i][-1] < fundos[i][-2]):
                df['TENDENCIA'][i] = 'BAIXA'
            if (((topos[i][-1] < topos[i][-2]) & (fundos[i][-1] > fundos[i][-2])) | ((topos[i][-1] > topos[i][-2]) & (fundos[i][-1] < fundos[i][-2]))):
                df['TENDENCIA'][i] = 'INDEFINIDA'

        else:
            df['TENDENCIA'][i] = 'INDEFINIDA'
    return(df,aux)   
        
        
    

#calculo da MME9
#tend = pd.DataFrame();
def MME9(df,aux):
    for i in range(df.shape[0]):
        aux[i].insert(aux[i].shape[1],'MME9',aux[i].Close.ewm(span=9).mean().dropna()) 
        aux[i].insert(aux[i].shape[1],'SLOPE',0.0)
        for j in range (1,aux[i].shape[0]):
            tend = aux[i].MME9[j] - aux[i].MME9[j-1]
            aux[i].iat[j,aux[i].shape[1]-1]=tend
    return(aux)

#função para deletar colunas extras
# aux = delcol(df,aux)
def delcol(df,aux):
    for i in range(df.shape[0]):
        aux[i].drop(columns=['TOPO','FUNDO','MME9','SLOPE','mark_min','mark_max'],errors='ignore',inplace = True)
    return(aux)

#retorna index do ticker
def index(ticker):
    index = (df[df['tickers'] == ticker].index.values)
    return index
# função setup 9.2 #1
    
def setup92(df,aux):    
    
    #perca de minima
    for i in range(df.shape[0]):
        aux[i]['mark_min'] = ''
        for j in range(1,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_min'] = np.where((((aux[i].Close[j] < aux[i].Low[j-1]) |(aux[i].Close[j] < aux[i].Open[j-1])) & (aux[i].SLOPE[j] > 0)),1,0)

    #superação de maxima
    for i in range(df.shape[0]):
        aux[i]['mark_max'] = ''
        for j in range(3,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_max'] = np.where(((aux[i].Close[j] > aux[i].High[j-1]) | (aux[i].Open[j] > aux[i].High[j-1])) & ((aux[i].mark_min[j] == 1) | (aux[i].mark_min[j-1] == 1) | (aux[i].mark_min[j-2] == 1) | (aux[i].mark_min[j-3] == 1)) ,1,0)

    #quantos mark_min
    contador_min=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_min[j] == 1):
                contador_min = contador_min + 1;

    #saber quantos mark_max
    contador_max=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_max[j] == 1):
                contador_max = contador_max + 1;

    #pegar todos os tickers com mark_max e as datas
    tickersmax = [];
    datamax = [];
    tickersmin = [];
    datamin = [];
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):

            if (aux[i]['mark_max'][j] == 1):
                tickersmax.append(df.iloc[i,0]);
                datamax.append(aux[i].index[j]);
            if (aux[i]['mark_min'][j] == 1):
                tickersmin.append(df.iloc[i,0]);
                datamin.append(aux[i].index[j]);
    #alocar os resultados em dois pd            
    df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
    df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
    #tirar valores duplicados ficando com o mais recente

    df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
    df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
    #setup 9.2 armado          
    df_setup_92_armado = pd.DataFrame();
    duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
    df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]
    
    #tirar os valores antigos >5 dias
    for i in range(df_setup_92_armado.shape[0]-1,-1,-1):
        diff = abs(df_setup_92_armado['datas'].iloc[-1] - df_setup_92_armado.datas.iloc[i]);
        if diff.days > 5:
            df_setup_92_armado = df_setup_92_armado.drop([df_setup_92_armado.index[i]])
    #reindexar o indice       
    
    df_setup_92_armado.index = range(df_setup_92_armado.shape[0])
    df_setup_92_armado.tickers.to_csv('setup92.csv',header=False, index=False,line_terminator=',')
    
    return

def setup92wk(df,aux):    
    
    #perca de minima
    for i in range(df.shape[0]):
        aux[i]['mark_min'] = ''
        for j in range(1,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_min'] = np.where((((aux[i].Close[j] < aux[i].Low[j-1]) |(aux[i].Close[j] < aux[i].Open[j-1])) & (aux[i].SLOPE[j] > 0)),1,0)

    #superação de maxima
    for i in range(df.shape[0]):
        aux[i]['mark_max'] = ''
        for j in range(3,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_max'] = np.where(((aux[i].Close[j] > aux[i].High[j-1]) | (aux[i].Open[j] > aux[i].High[j-1])) & ((aux[i].mark_min[j] == 1) | (aux[i].mark_min[j-1] == 1) | (aux[i].mark_min[j-2] == 1) | (aux[i].mark_min[j-3] == 1)) ,1,0)

    #quantos mark_min
    contador_min=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_min[j] == 1):
                contador_min = contador_min + 1;

    #saber quantos mark_max
    contador_max=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_max[j] == 1):
                contador_max = contador_max + 1;
    #pegar todos os tickers com mark_max e as datas
    tickersmax = [];
    datamax = [];
    tickersmin = [];
    datamin = [];
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):

            if (aux[i]['mark_max'][j] == 1):
                tickersmax.append(df.iloc[i,0]);
                datamax.append(aux[i].index[j]);
            if (aux[i]['mark_min'][j] == 1):
                tickersmin.append(df.iloc[i,0]);
                datamin.append(aux[i].index[j]);
    #alocar os resultados em dois pd            
    df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
    df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
    #tirar valores duplicados ficando com o mais recente

    df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
    df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
    #setup 9.2 armado          
    df_setup_92_armado = pd.DataFrame();
    duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
    df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]
    
    #tirar os valores antigos >5 dias
    for i in range(df_setup_92_armado.shape[0]-1,-1,-1):
        diff = abs(df_setup_92_armado['datas'].iloc[-1] - df_setup_92_armado.datas.iloc[i]);
        if diff.days > 20:
            df_setup_92_armado = df_setup_92_armado.drop([df_setup_92_armado.index[i]])
    #reindexar o indice       
    
    df_setup_92_armado.index = range(df_setup_92_armado.shape[0])
    df_setup_92_armado.tickers.to_csv('setup92wk.csv',header=False, index=False,line_terminator=',')
    
    return
def setup92mo(df,aux):    
    
    #perca de minima
    for i in range(df.shape[0]):
        aux[i]['mark_min'] = ''
        for j in range(1,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_min'] = np.where((((aux[i].Close[j] < aux[i].Low[j-1]) |(aux[i].Close[j] < aux[i].Open[j-1])) & (aux[i].SLOPE[j] > 0)),1,0)

    #superação de maxima
    for i in range(df.shape[0]):
        aux[i]['mark_max'] = ''
        for j in range(3,aux[i].shape[0]):
            aux[i].loc[aux[i].index[j],'mark_max'] = np.where(((aux[i].Close[j] > aux[i].High[j-1]) | (aux[i].Open[j] > aux[i].High[j-1])) & ((aux[i].mark_min[j] == 1) | (aux[i].mark_min[j-1] == 1) | (aux[i].mark_min[j-2] == 1) | (aux[i].mark_min[j-3] == 1)) ,1,0)

    #quantos mark_min
    contador_min=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_min[j] == 1):
                contador_min = contador_min + 1;

    #saber quantos mark_max
    contador_max=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_max[j] == 1):
                contador_max = contador_max + 1;
    #pegar todos os tickers com mark_max e as datas
    tickersmax = [];
    datamax = [];
    tickersmin = [];
    datamin = [];
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):

            if (aux[i]['mark_max'][j] == 1):
                tickersmax.append(df.iloc[i,0]);
                datamax.append(aux[i].index[j]);
            if (aux[i]['mark_min'][j] == 1):
                tickersmin.append(df.iloc[i,0]);
                datamin.append(aux[i].index[j]);
    #alocar os resultados em dois pd            
    df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
    df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
    #tirar valores duplicados ficando com o mais recente

    df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
    df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
    #setup 9.2 armado          
    df_setup_92_armado = pd.DataFrame();
    duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
    df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]
    
    #tirar os valores antigos >5 dias
    for i in range(df_setup_92_armado.shape[0]-1,-1,-1):
        diff = abs(df_setup_92_armado['datas'].iloc[-1] - df_setup_92_armado.datas.iloc[i]);
        if diff.days > 60:
            df_setup_92_armado = df_setup_92_armado.drop([df_setup_92_armado.index[i]])
    #reindexar o indice       
    
    df_setup_92_armado.index = range(df_setup_92_armado.shape[0])
    df_setup_92_armado.tickers.to_csv('setup92mo.csv',header=False, index=False,line_terminator=',')
    
    return
#Linha de avanço e declinio
# LAD(df,aux,LADdf)
def LAD(df,aux,LADdf):
    a = datetime.datetime.now();
    lad = 0;
    if a.hour >= 17:
        for i in range(df.shape[0]):
            if aux[i].Close[aux[i].shape[0]-1] > aux[i].Open[aux[i].shape[0]-1]:
                lad += 1
            if aux[i].Close[aux[i].shape[0]-1] < aux[i].Open[aux[i].shape[0]-1]:
                lad -=1
    print(lad)
    LADdf.append(lad)
    return(LADdf)    
                
                

#gappingup(df,aux)
def gappingup (df,aux):
    gap_up = []
    for i in range(df.shape[0]):
        if aux[i].index[aux[i].shape[0]-1] == date.today():
            if ((aux[i].Open[aux[i].shape[0]-1] / aux[i].Close[aux[i].shape[0]-2]) > 1.01):
                gap_up.append(df.iloc[i,0])
    gap_up = pd.Series(gap_up)

    gap_up.to_csv('gap_up.csv',header=False, index=False,line_terminator=',')
    return
#gappingdown(df,aux)
def gappingdown (df,aux):
    gap_down = []
    for i in range(df.shape[0]):
        if aux[i].index[aux[i].shape[0]-1] == date.today():
                if ((aux[i].Open[aux[i].shape[0]-1] / aux[i].Close[aux[i].shape[0]-2]) < 0.99):
                    gap_down.append(df.iloc[i,0])
    gap_down = pd.Series(gap_down)

    gap_down.to_csv('gap_down.csv',header=False, index=False,line_terminator=',')
    return

In [15]:
df,aux,auxwk,auxmo = carregardf();

In [237]:
df['TENDENCIA'][0] = 'caqui'

In [12]:
date.today().weekday()

3

In [19]:
df.iat[0,1] = aux[0];

NameError: name 'i' is not defined

In [281]:
df.loc[lambda df: df['TENDENCIA'] == 'BAIXA']

,tickers,dados,dadoswe,dadosmo,TENDENCIA


In [271]:
for i in range(df.shape[0]):
    auxwk[i].dropna(inplace=True)

In [294]:

#index('WIZS3')
auxwk[267].tail(20)

,Open,High,Low,Close,Adj Close,Volume,TOPO,FUNDO
Date,,,,,,,,
2021-01-03,8.05,8.12,7.68,7.99,7.434648,5550700.0,0,0
2021-01-10,7.94,7.96,7.62,7.88,7.332294,6437100.0,0,0
2021-01-17,7.92,7.96,7.62,7.74,7.202024,5191900.0,0,0
2021-01-24,7.74,8.02,7.66,7.67,7.136889,4545200.0,0,0
2021-01-31,7.78,8.61,7.67,8.35,7.769626,8490500.0,1,0
2021-02-07,8.39,8.47,7.68,7.84,7.295074,5551700.0,0,0
2021-02-14,7.84,7.84,6.30,6.33,5.890028,13131300.0,0,0
2021-02-22,6.27,6.93,6.05,6.26,5.824893,11756200.0,0,1
2021-03-01,6.35,7.22,6.13,6.47,6.020297,20263400.0,1,0


In [272]:
auxwk = toposefundos(df,auxwk)

In [299]:
%%capture
aux = atualizacaodiaria(df,aux)

BAIXAR DADOS df,aux = baixardados(ndias);

In [235]:
df = lercsv('B3.csv')
df,aux = baixardados(df,15)
winsound.Beep(690,420)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [42]:
LADdf = []

In [44]:
LADdf = LAD(df,aux,LADdf)

-16


In [45]:
LADdf

[-16]

In [300]:
%%capture
t1 = time();
#df,aux = carregardf();
#aux = atualizacaodiaria(df,aux);
#auxwk = MME9(df,auxwk);
#auxmo = MME9(df,auxmo);
aux = MME9(df,aux);
setup92(df,aux);
#setup92wk(df,auxwk);
#setup92mo(df,auxmo);
#gappingup(df,auxmo);
#gappingdown(df,auxmo);
t2 = time() - t1;
winsound.Beep(690,500)
print('Tempo decorrido: {} segundos'.format(t2))

RETORNA INDEX DO TICKER print(index('ticker'))

In [5]:
#retorna index em dataframe
#print(df_setup_92_armado[df_setup_92_armado['tickers'] == 'ALSO3'].index.values)


In [166]:
index('BPAC11')

array([39], dtype=int64)

In [ ]:
#preços intraday
for i in range(df_setup_92_armado.shape[0]):
    for j in range(df.shape[0]):
        if (df_setup_92_armado.iloc[i,1]) == (df.iloc[j,0]):
            if intradayaux[i].shape[0] != 0:
                if (intradayaux[i].Close[intradayaux[i].shape[0]-1]) >= (aux[j].High[aux[j].shape[0]-1]) :
                    print (df_setup_92_armado.iloc[i,1])
                    
#watchlist
intradayaux = []
#if (df_setup_92_armado['dados']):
  #  del df_setup_92_armado['dados']
for i in range(df_setup_92_armado.shape[0]):
    #startdate = df_setup_92_armado.iloc[i,0];
    today = date.today();
    tomorrow = date.today() + datetime.timedelta(days=1)
    intradayaux.append(pdr.get_data_yahoo(df_setup_92_armado.iloc[i,1]+'.sa',start= today,end = tomorrow,interval="5m"));
    
df_setup_92_armado.insert(2, 'dados', intradayaux, allow_duplicates=True);

df_setup_92_armado.index = range(df_setup_92_armado.shape[0])

print('======= ACABOU ===========')